<a href="https://colab.research.google.com/github/Hoku113/Federated-GPT2-finetune/blob/main/Federated_GPT2-finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Federated Learning LLM finetune

In [ ]:
%cd drive/MyDrive/finetune-gpt/
%pwd

/content/drive/MyDrive/finetune-gpt


'/content/drive/MyDrive/finetune-gpt'

In [ ]:
!pip install --upgrade pip
!pip install flwr[simulation] flwr_datasets[vision] transformers datasets evaluate
!pip install Cython evaluate
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 16.5 MB

In [ ]:
!nvidia-smi

Thu Feb 22 15:31:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [19]:
# Load model directly
import torch
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM,TextDataset,DataCollatorForLanguageModeling

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

tokenizer.save_pretrained("./model")


# set default device and number of clients
NUM_CLIENTS = 2
DEVICE = torch.device("cuda")

def load_dataset(train_dataset, test_dataset, tokenizer):

  # データセットに格納する
  train_dataset = TextDataset(
      tokenizer=tokenizer,
      file_path=train_dataset,
      block_size=128
  )

  test_dataset = TextDataset(
      tokenizer=tokenizer,
      file_path=test_dataset,
      block_size=128
  )

  data_collator = DataCollatorForLanguageModeling(
      tokenizer=tokenizer, mlm=False
  )

  return train_dataset, test_dataset, data_collator

# ダウンロードしたデータをテキストファイルに一度変換する
def build_text_files(data_json, dest_path):
  f = open(dest_path, 'w')
  data = ''
  for texts in data_json:
    summary = str(texts["instruction"]).strip()
    summary = re.sub(r"\s", " ", summary)
    data += summary + " "
  f.write(data)

  return dest_path

model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

In [ ]:
train_path = "train_dataset.txt"
test_path = "test_dataset.txt"

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
print(f"train_dataset: {train_dataset}")
print(f"test_dataset: {test_dataset}")
print(f"train_dataset length: {len(train_dataset)}")
print(f"test_dataset length: {len(test_dataset)}")

train_dataset: <transformers.data.datasets.language_modeling.TextDataset object at 0x7d3dfbba30a0>
test_dataset: <transformers.data.datasets.language_modeling.TextDataset object at 0x7d3cd8e144c0>
train_dataset length: 755698
test_dataset length: 755698


In [ ]:
inputnum = len(train_dataset)
i =2
while True:
  print(inputnum / i)

  if inputnum % i == 0:
    print(i)
    break

  i += 1

377849.0
2


In [ ]:
import random
import torch
import re
import json
from torch.utils.data import DataLoader, random_split

def build_dataloader(num_clients: int, train_dataset):

  partition_size = len(train_dataset) // num_clients
  lengths = [partition_size] * num_clients

  print(train_dataset)
  print(sum(lengths))

  datasets = random_split(train_dataset, lengths, torch.Generator().manual_seed(42))

  trainloaders = []
  valloaders = []

  for ds in datasets:
    len_val = len(ds) // num_clients
    len_train = len(ds) - len_val
    lengths = [len_train, len_val]
    ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
    trainloaders.append(ds_train)
    valloaders.append(ds_val)

  return trainloaders, valloaders

trainloaders, valloaders = build_dataloader(NUM_CLIENTS, train_dataset)

print(trainloaders)
print(valloaders)

755698
[<torch.utils.data.dataset.Subset object at 0x7d3cd8cd68c0>, <torch.utils.data.dataset.Subset object at 0x7d3dfbb6b6a0>]
[<torch.utils.data.dataset.Subset object at 0x7d3cd8cd6860>, <torch.utils.data.dataset.Subset object at 0x7d3dfbba19f0>]


In [ ]:
from evaluate import load as load_metric
from transformers import AdamW
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

def train(net, trainloader, data_collator, testloader, epochs):

  print(type(trainloader))

  training_args = TrainingArguments(
      output_dir="./output",
      overwrite_output_dir=True,
      num_train_epochs=epochs,
      per_device_train_batch_size=64,
      logging_steps=100,
      save_steps=800
  )

  trainer = Trainer(
      model=net,
      args=training_args,
      data_collator=data_collator,
      train_dataset=trainloader
  )

  trainer.train()
  trainer.save_model(output_dir="model")
  print("saved model!")


  # optimizer = AdamW(net.parameters(), lr=5e-5)
  # net.train()
  # for _ in range(epochs):
  #   for batch in trainloader:
  #       # print(batch)
  #       # batch = {k: v.to(DEVICE) for k, v in batch.items()}
  #       batch.to(DEVICE)
  #       outputs = net(**batch)
  #       loss = outputs.loss
  #       loss.backward()
  #       optimizer.step()
  #       optimizer.zero_grad()

def test(testloader):
  # training_args = TrainingArguments(
  #     output_dir="./output",
  #     overwrite_output_dir=True,
  #     num_train_epochs=1,
  #     per_device_train_batch_size=32,
  #     logging_steps=100,
  #     save_steps=800
  # )

  # trainer = Trainer(
  #   model=net,
  #   args=training_args,
  #   data_collator=data_collator,
  #   train_dataset=testloader
  # )

  trainer.test(dataloaders=testloader)


In [ ]:
# Creating the model itself
from transformers import AutoModelForCausalLM
# net = AutoModelForCausalLM.from_pretrained("gpt2")
net = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")


In [ ]:
from collections import OrderedDict
import flwr as fl

class GPTClient(fl.client.NumPyClient):

  def __init__(self, net, trainloader, valloader, data_collator):
    self.net = net
    self.trainloader = trainloader
    self.valloader = valloader
    self.data_collator = data_collator

  def get_parameters(self, config):
    return [val.cpu().numpy() for _, val in self.net.state_dict().items()]

  def set_parameters(self, parameters):
    params_dict = zip(self.net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    self.net.load_state_dict(state_dict, strict=True)

  def fit(self, parameters, config):
    self.set_parameters(parameters)
    print("Training Started...")
    self.trainer = train(self.net, self.trainloader, self.data_collator, self.valloader, epochs=1 )
    print("Training Finished.")
    return self.get_parameters(config={}), len(self.trainloader), {}

  # def evaluate(self, parameters, config):
  #   print(f"self trainer: {self.trainer}")
  #   self.set_parameters(parameters)
  #   loss, accuracy = test(self.trainer, self.valloader)
  #   return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


In [ ]:
def client_fn(cid: str) -> GPTClient:
    # net = AutoModelForCausalLM.from_pretrained("gpt2").to(DEVICE)
    # net = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium").to(DEVICE)

    # それぞれのクライアントにデータを割り当てる
    trainloader = trainloaders[int(cid)]
    testloader = valloaders[int(cid)]
    return GPTClient(model, trainloader, testloader, data_collator).to_client()

def weighted_average(metrics):
  accuracies = [num_examples * m["accuracy"] for num_exmaples, m in metrics]
  losses = [num_examples * m["loss"] for num_examples, m in metrics]
  examples = [num_examples for num_examples, _ in metrics]
  return {"accuracy": sum(accuracies) / sum(examples), "loss": sum(losses) / sum(examples)}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    evaluate_metrics_aggregation_fn=weighted_average,
)

client_resources = {"num_cpus" : 1, "num_gpus": 0.0}
if DEVICE.type == "cuda":
    # here we are asigning an entire GPU for each client.
    client_resources = {"num_cpus": 1, "num_gpus": 1.0}
    # Refer to our documentation for more details about Flower Simulations
    # and how to setup these `client_resources`.

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=1),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2024-02-21 13:33:38,727 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)
2024-02-21 13:33:41,590	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-02-21 13:33:43,032 | app.py:213 | Flower VCE: Ray initialized with resources: {'object_store_memory': 26741036236.0, 'accelerator_type:A100': 1.0, 'CPU': 12.0, 'memory': 53482072475.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 26741036236.0, 'accelerator_type:A100': 1.0, 'CPU': 12.0, 'memory': 53482072475.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO flwr 2024-02-21 13:33:43,035 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower V

(DefaultActor pid=2028) Training Started...
(DefaultActor pid=2028) <class 'torch.utils.data.dataset.Subset'>


  3%|▎         | 100/2952 [01:57<54:57,  1.16s/it]


(DefaultActor pid=2028) {'loss': 0.5398, 'learning_rate': 4.8306233062330626e-05, 'epoch': 0.03}


  7%|▋         | 200/2952 [03:52<53:01,  1.16s/it]


...

 98%|█████████▊| 2900/2952 [58:28<01:00,  1.16s/it]


(DefaultActor pid=2028) {'loss': 0.355, 'learning_rate': 8.807588075880759e-07, 'epoch': 0.98}


100%|██████████| 2952/2952 [59:28<00:00,  1.21s/it]


(DefaultActor pid=2028) {'train_runtime': 3568.9966, 'train_samples_per_second': 52.935, 'train_steps_per_second': 0.827, 'train_loss': 0.37628115418803726, 'epoch': 1.0}
(DefaultActor pid=2028) saved model!
(DefaultActor pid=2028) Training Finished.


DEBUG flwr 2024-02-21 15:32:24,298 | server.py:236 | fit_round 1 received 2 results and 0 failures
DEBUG:flwr:fit_round 1 received 2 results and 0 failures
WARNING flwr 2024-02-21 15:32:27,305 | fedavg.py:250 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-02-21 15:32:27,325 | server.py:173 | evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG:flwr:evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2024-02-21 15:32:52,417 | server.py:187 | evaluate_round 1 received 0 results and 2 failures
DEBUG:flwr:evaluate_round 1 received 0 results and 2 failures
INFO flwr 2024-02-21 15:32:52,419 | server.py:153 | FL finished in 7127.6742736510005
INFO:flwr:FL finished in 7127.6742736510005
INFO flwr 2024-02-21 15:32:52,517 | app.py:226 | app_fit: losses_distributed []
INFO:flwr:app_fit: losses_distributed []
INFO flwr 2024-02-21 15:32:52,518 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2024-02-21 15:32

学習したモデルを使ってみる

In [9]:
# ディレクトリの移動 (学習を行った場合は不要)
%cd drive/MyDrive/finetune-gpt/
%pwd

[Errno 2] No such file or directory: 'drive/MyDrive/finetune-gpt/'
/content/drive/MyDrive/finetune-gpt


'/content/drive/MyDrive/finetune-gpt'

In [10]:
!pip install --upgrade pip
!pip install flwr[simulation] flwr_datasets[vision] transformers[sentencepiece] datasets evaluate
!pip install Cython evaluate
!pip install transformers[torch]
!pip install accelerate -U

In [20]:
import torch
from transformers import AutoTokenizer, T5TokenizerFast,AutoModelForCausalLM, GPT2LMHeadModel, GPT2TokenizerFast

device = "cuda" if torch.cuda.is_available() else "cpu"

# トークナイザとモデルを読み込む
tokenizer = AutoTokenizer.from_pretrained("./model/")
model = GPT2LMHeadModel.from_pretrained("./model/").to(device)

print(f"tokenizer: {tokenizer}")
print(f"model: {model}")

tokenizer: T5TokenizerFast(name_or_path='./model/', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, n

In [22]:
# dir(tokenizer)
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_auto_class',
 '_autoset_attn_implementation',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_check_and_enable_flash_attn_2',
 '_check_and_enable_sdpa',
 '_compiled_call_impl',
 '_convert_head_mask_to_5d',
 '_copy_lm_head_original_to_resized',
 '_create_repo',
 '_dispatch_accelerate_model',
 '_expand_inputs_for_generation',
 '_extract_past_from_model_output',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_k

In [50]:
# 推論させたい言葉を入力する
text = '自然言語処理の学習を始めるには何から始めるできでしょうか？　実は人によって'

# text = input("会話の文章を入力: ")

# トークナイザーを使用してモデルへの入力に変換
input = tokenizer.encode(text, return_tensors='pt', add_special_tokens=False).to(device)

# 文章を生成する
output = model.generate(input, max_length=100, min_length=100, eos_token_id=tokenizer.eos_token_id ,bos_token_id=tokenizer.bos_token_id ,pad_token_id=tokenizer.pad_token_id, early_stopping=True)

print(output)

decoded = tokenizer.decode(output[0])
print(decoded)

tensor([[    9,  1129,  1040,  1465,    10,  3771,  4732,    42,  1059,    28,
          4880,  3092, 16744,    95,  3017,     9,  7754,    63,    73,     7,
            65,    63,    10,  1040,    10,  4795,    73,     7,    65,    63,
            10,  1040,    10,  4795,    17,  4277,   498,     7,    65,  1040,
            18,  3771,    35,    10,    17, 16464,    57,  1180,    12, 17180,
           341,   203,  3418,     8,     9,  1497,  1036,    18,    99, 26341,
            55, 19120,     8,     9,  1497,  1036,    18,    99, 26341,    55,
         19120,     8,     9,  1497,  1036,    18,    99, 26341,    55, 19120,
             8,     9,  1497,  1036,    18,    99, 26341,    55, 19120,     8,
             9,  1497,  1036,    18,    99, 26341,    55, 19120,     8,     9]],
       device='cuda:0')
自然言語処理の学習を始めるには何から始めるできでしょうか? 実は人によって、その人の言語の特性によって、その人の言語の特性に合うように、その言語を学習するのに最適な方法が違ってきます。 次の英語を日本語に翻訳してください。 次の英語を日本語に翻訳してください。 次の英語を日本語に翻訳してください。 次の英語を日本語に翻訳してください。 次の英語を日本語に翻訳してくだ